In [ ]:

iterables = [ ['heat', 'wave'], ['complete', None, 'empty'] ]  
for t in itertools.product(*iterables):
    print(t)
    for file_name in ['PTC_MR']:
        graph_list, data_y, data_nodes  = return_dataset(file_name)
        feature_matrix = []
        for A in graph_list:
            feature_matrix.append(netlsd.netlsd(A, kernel=t[0], eigenvalues='full', normalization=t[1], normalized_laplacian=False))
        np.savetxt(file_name+"_"+t[0]+"_"+str(t[1])+"_NETLSD.txt",feature_matrix,delimiter=',')

In [9]:
#import igraph
from sklearn.ensemble import RandomForestClassifier
import pickle, netlsd, itertools
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.utils import shuffle
np.random.seed(1656) 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import pickle
import networkx as nx

In [4]:
data = pd.read_pickle('new_data_nx')

In [32]:
data_X = data['graph'].apply(lambda x: netlsd.netlsd(nx.adjacency_matrix(x).todense(), kernel='heat', eigenvalues='full', normalization='complete', normalized_laplacian=False))
data_y = data['target'].values

In [38]:
data_X = data_X.values

In [58]:
data_X1 = [np.nan_to_num(np.array(i), nan=0, posinf=0, neginf=0) for i in data_X]

In [62]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
data_X1 = min_max_scaler.fit_transform(data_X1)

In [54]:
data_X = list([list(i) for i in data_X])

In [63]:
def return_grid_search_random_forest():
    return RandomForestClassifier(n_estimators=500, 
                        min_samples_leaf=1, 
                            min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, bootstrap=True, 
                                oob_score=False, n_jobs=8, random_state=666, verbose=0, warm_start=False,
                                    max_depth = None,class_weight="balanced_subsample")


#graph_list, data_y, data_nodes  = return_dataset('NCI1')
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state = 42)
estimator  = RandomForestClassifier(criterion='gini', max_depth=None, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                        max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, 
                                        oob_score=False, n_jobs=8, random_state=666, verbose=0, warm_start=False,
                                        class_weight=None)

param_grid = {'n_estimators':[100,500,1000], 'max_features':['sqrt'], 'min_samples_split':[3,4]}

grid_rf    = GridSearchCV(estimator, param_grid, scoring=None, n_jobs=8,
                    refit=True, cv=kf, verbose=3, pre_dispatch='2*n_jobs')

#print(data_X.shape)
grid_rf.fit(data_X1, data_y)
#grid_rf.cv_results_['mean_test_score']
#Pipeline2 = Pipeline([('drop_empty_column', dropCols(frac=1.0)),('grid_rf', grid_rf)])
#res = Pipeline2.fit(data_X, data_y)
print( grid_rf.cv_results_['mean_test_score'])
print('*'*20)
print(grid_rf.cv_results_['std_test_score'])
print('*'*20)
print('max: ',max( grid_rf.cv_results_['mean_test_score']))
print('*'*25, '\n\n')
#print(np.concatenate(feature_matrix).ravel().max())
#c = sparse.vstack(feature_matrix)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[0.4282752  0.42975635 0.42960824 0.43049693 0.429312   0.4287936 ]
********************
[0.0081011  0.00932303 0.00888938 0.00878512 0.00808077 0.00897718]
********************
max:  0.43049692660890176
************************* 


[CV 2/3] END max_features=sqrt, min_samples_split=3, n_estimators=100;, score=0.420 total time=   8.2s
[CV 1/3] END max_features=sqrt, min_samples_split=4, n_estimators=100;, score=0.443 total time=   7.4s
[CV 3/3] END max_features=sqrt, min_samples_split=4, n_estimators=100;, score=0.426 total time=   9.9s
[CV 2/3] END max_features=sqrt, min_samples_split=4, n_estimators=500;, score=0.420 total time=  46.5s
[CV 1/3] END max_features=sqrt, min_samples_split=3, n_estimators=1000;, score=0.441 total time= 1.2min
[CV 1/3] END max_features=sqrt, min_samples_split=3, n_estimators=500;, score=0.442 total time=  35.8s
[CV 3/3] END max_features=sqrt, min_samples_split=4, n_estimators=500;, score=0.429 tota

In [45]:
GridSearchCV?